<a href="https://colab.research.google.com/github/karielson/PPGEP9002_INTELIGENCIA_COMPUTACIONAL/blob/main/Classifica%C3%A7%C3%A3o_de_Textos_usando_Redes_Neurais_e_M%C3%A9todos_de_Classifica%C3%A7%C3%A3o_Supervisionada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Textos usando Redes Neurais e Métodos Supervisionados

**Aluno:** Karielson Medeiros Feitosa  
**Disciplina:**  PPGEP9002 - 2024.2  
**Professor:** José Alfredo F. Costa  

---

## 1. Introdução
Esta tarefa tem como objetivo aplicar diferentes métodos de classificação de textos utilizando redes neurais e técnicas de aprendizado supervisionado. A base de dados utilizada contém textos categorizados em 6 classes diferentes, abrangendo diversas áreas temáticas.

### Objetivo
Explorar diferentes representações textuais e arquiteturas de redes neurais para comparar o desempenho dos modelos no problema de classificação de textos.



---

## 2. Pré-Processamento de Textos

Nesta seção, serão realizadas as seguintes etapas de pré-processamento:
1. Conversão para minúsculas.
2. Remoção de pontuações e caracteres especiais.
3. Remoção de stopwords.
4. Tokenização e lematização.
5. Divisão dos dados em treino, validação e teste (70%-15%-15%).


In [25]:
# Importar bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from transformers import BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset

# Baixar recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Adicionado para resolver dependência de lematização

# Carregar a base de dados (substituir 'file_path' pelo caminho do arquivo no Google Colab)
file_path = '/content/Base_dados_textos_6_classes.csv'
df = pd.read_csv(file_path, sep=';', encoding='latin1')  # Corrigir encoding para 'latin1' para lidar com caracteres especiais

# Exibir informações da base de dados
print("Informações da base de dados:")
print(df.info())
print(df.head())

# Remover linhas com valores nulos
print("Removendo linhas com valores nulos...")
df.dropna(subset=['Texto Original', 'Classe'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Função para limpeza do texto
def preprocess_text(text):
    if pd.isnull(text):  # Verificar se o texto é nulo
        return ''
    text = text.lower()  # Converter para minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remover pontuações e caracteres especiais
    tokens = word_tokenize(text)  # Tokenizar
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]  # Lematizar e remover stopwords
    return ' '.join(tokens)

# Aplicar o pré-processamento
try:
    df['Texto Limpo'] = df['Texto Original'].apply(preprocess_text)
except LookupError as e:
    print(f"Erro durante o pré-processamento: {e}")
    nltk.download('punkt_tab')  # Adicionando tentativa de baixar recurso adicional
    df['Texto Limpo'] = df['Texto Original'].apply(preprocess_text)

# Verificar se todas as colunas necessárias estão presentes
if 'Texto Limpo' in df.columns and 'Classe' in df.columns:
    # Dividir a base de dados em treino, validação e teste (70%-15%-15%)
    train_data, temp_data, train_labels, temp_labels = train_test_split(df['Texto Limpo'], df['Classe'], test_size=0.30, random_state=42)
    val_data, test_data, val_labels, test_labels = train_test_split(temp_data, temp_labels, test_size=0.50, random_state=42)
else:
    raise KeyError("As colunas 'Texto Limpo' e 'Classe' são necessárias para a divisão dos dados e não foram encontradas.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Informações da base de dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Texto Original   315 non-null    object
 1   Texto Expandido  315 non-null    object
 2   Classe           319 non-null    int64 
 3   Categoria        319 non-null    object
dtypes: int64(1), object(3)
memory usage: 10.1+ KB
None
                                      Texto Original  \
0  Desenvolvimento de criptomoedas e blockchain: ...   
1  Economia colaborativa: plataformas que revoluc...   
2  Economia criativa no audiovisual: potencial de...   
3  Economia do conhecimento: investimentos em edu...   
4  Indústria 4.0 no Brasil: transformação digital...   

                                     Texto Expandido  Classe Categoria  
0  O Brasil emerge como um polo de inovação no me...       0  Economia  
1  Plataformas de economia colaborativa estão red...     

---

## 3. Geração de Representações Textuais

Nesta etapa, utilizaremos três técnicas de geração de embeddings:
1. **TF-IDF**
2. **Word2Vec**
3. **Transformers (BERT)**

### 3.1 TF-IDF

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Criar embeddings usando TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(train_data)
X_val_tfidf = vectorizer.transform(val_data)
X_test_tfidf = vectorizer.transform(test_data)


### 3.2 Word2Vec

In [14]:
from gensim.models import Word2Vec

# Tokenizar os textos para Word2Vec
tokenized_train = [text.split() for text in train_data]

# Treinar o modelo Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_train, vector_size=100, window=5, min_count=1, workers=4)

# Função para gerar embeddings a partir do Word2Vec
def get_word2vec_embeddings(data, model):
    embeddings = []
    for text in data:
        words = text.split()
        word_embeddings = [model.wv[word] for word in words if word in model.wv]
        if word_embeddings:
            embeddings.append(sum(word_embeddings) / len(word_embeddings))
        else:
            embeddings.append([0] * model.vector_size)
    return embeddings

X_train_w2v = get_word2vec_embeddings(train_data, word2vec_model)
X_val_w2v = get_word2vec_embeddings(val_data, word2vec_model)
X_test_w2v = get_word2vec_embeddings(test_data, word2vec_model)

### 3.3 Transformers (BERT)

In [26]:
from transformers import BertTokenizer, BertModel
import torch

# Carregar modelo e tokenizer BERT
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Função para gerar embeddings usando BERT
def get_bert_embeddings(data):
    embeddings = []
    for text in data:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        outputs = bert_model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())
    return embeddings

X_train_bert = get_bert_embeddings(train_data)
X_val_bert = get_bert_embeddings(val_data)
X_test_bert = get_bert_embeddings(test_data)

---

## 4. Modelos de Classificação

Nesta seção, implementaremos três modelos de classificação:
1. **MLP (Multilayer Perceptron)**
2. **CNN (Redes Convolucionais)**
3. **Transformers (Fine-Tuning de BERT)**

### 4.1 MLP

In [17]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Treinar o modelo MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp.fit(X_train_tfidf, train_labels)

# Avaliar o modelo
predictions = mlp.predict(X_test_tfidf)
print("Acurácia:", accuracy_score(test_labels, predictions))
print("Relatório de Classificação:\n", classification_report(test_labels, predictions))
print("Matriz de confusão:\n", confusion_matrix(test_labels, predictions))

Acurácia: 0.75
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.86      0.75      0.80         8
           1       0.70      0.88      0.78         8
           2       0.75      0.43      0.55         7
           3       0.62      0.62      0.62         8
           4       0.75      1.00      0.86         9
           5       0.86      0.75      0.80         8

    accuracy                           0.75        48
   macro avg       0.76      0.74      0.73        48
weighted avg       0.76      0.75      0.74        48

Matriz de confusão:
 [[6 1 0 0 1 0]
 [1 7 0 0 0 0]
 [0 0 3 3 0 1]
 [0 1 1 5 1 0]
 [0 0 0 0 9 0]
 [0 1 0 0 1 6]]


### 4.2 CNN

In [18]:
cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=5000, output_dim=100, input_length=X_train_tfidf.shape[1]))
cnn_model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(10, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

cnn_model.fit(np.array(X_train_tfidf.todense()), np.array(train_labels), epochs=5, batch_size=32, validation_data=(np.array(X_val_tfidf.todense()), np.array(val_labels)))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 745ms/step - accuracy: 0.1402 - loss: -0.3519 - val_accuracy: 0.1875 - val_loss: -4.8170
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 340ms/step - accuracy: 0.2000 - loss: -10.9282 - val_accuracy: 0.1875 - val_loss: -21.3331
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 381ms/step - accuracy: 0.1729 - loss: -43.0261 - val_accuracy: 0.1875 - val_loss: -61.2461
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 229ms/step - accuracy: 0.1718 - loss: -112.9620 - val_accuracy: 0.1875 - val_loss: -138.8782
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 315ms/step - accuracy: 0.1712 - loss: -224.0119 - val_accuracy: 0.1875 - val_loss: -272.0915


#### Avaliar CNN

In [19]:
cnn_predictions = cnn_model.predict(np.array(X_test_tfidf.todense()))
cnn_predictions = (cnn_predictions > 0.5).astype(int)
print("Acurácia CNN:", accuracy_score(test_labels, cnn_predictions))
print("Relatório de Classificação CNN:\n", classification_report(test_labels, cnn_predictions))
print("Matriz de confusão CNN:\n", confusion_matrix(test_labels, cnn_predictions))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Acurácia CNN: 0.16666666666666666
Relatório de Classificação CNN:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.17      1.00      0.29         8
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00         8
           4       0.00      0.00      0.00         9
           5       0.00      0.00      0.00         8

    accuracy                           0.17        48
   macro avg       0.03      0.17      0.05        48
weighted avg       0.03      0.17      0.05        48

Matriz de confusão CNN:
 [[0 8 0 0 0 0]
 [0 8 0 0 0 0]
 [0 7 0 0 0 0]
 [0 8 0 0 0 0]
 [0 9 0 0 0 0]
 [0 8 0 0 0 0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 4.3 Fine-Tuning de BERT

In [27]:
bert_fine_tune_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
optimizer = AdamW(bert_fine_tune_model.parameters(), lr=5e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
# Preparar os dados para DataLoader
def prepare_dataloader(data, labels, tokenizer, max_length=512, batch_size=16):
    inputs = tokenizer(list(data), return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], torch.tensor(labels))
    return DataLoader(dataset, batch_size=batch_size)

train_loader = prepare_dataloader(train_data, train_labels, tokenizer)
val_loader = prepare_dataloader(val_data, val_labels, tokenizer)
test_loader = prepare_dataloader(test_data, test_labels, tokenizer)

KeyError: 3